In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf
import pickle

In [6]:
with open('best_policy_random.pkl', 'rb') as f:
    gd = pickle.load(f)

In [7]:
gd[0][0]

'\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n          

In [8]:
blob ='\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n             | $$ $$\\$$\\$$| $$  | $$| $$$$$$$\\| $$      | $$  | $$\n             | $$$$  \\$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$\n             | $$$    \\$$$ \\$$    $$| $$  | $$| $$     \\| $$    $$\n              \\$$      \\$$  \\$$$$$$  \\$$   \\$$ \\$$$$$$$$ \\$$$$$$$ \n\n'
bloblen = len(blob)

gd[0] = [x.replace(blob, "") for x in gd[0]]

In [21]:
len_data = len(gd[0])

vocab_size = 3000
training_split = 0.7

traininglen = int(len_data * training_split)

# traininglen = 2000
training_set = gd[0][:traininglen]
test_set = gd[0][traininglen:]
test_set[0]


#tm = bowizer.TokenMaker(corpus=gd[0], vocab_size= vocab_size)



"\n-= Workshop =-\nYou've entered a workshop. You begin to take stock of what's in the room.\n\n\n\nThere is an exit to the north. Don't worry, it is unguarded. There is an exit to the west. Don't worry, it is unblocked.\n"

In [22]:
r = np.random.RandomState(42)

In [10]:
with open('norvig_tm_3000.pkl', 'rb') as f:
    tm = pickle.load(f)

In [11]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 256)
conv_h = ConvHyper(256, 6, 4)
rnn_h = RnnHyper(512, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [12]:
dernn_h = RnnHyper(512, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(256, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [13]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [14]:
np.array(training_set).shape

ml = 128

In [17]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [18]:
testmodel = make_model(ml)

In [19]:
testmodel.load_weights('../models/wl_model64')

In [31]:
def training_round(history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train = bowizer.SlicedWordData(lines=r.choice(training_set, size=2000, replace=False), maxlen=ml,tokenmaker=tm)
    test = bowizer.SlicedWordData(lines=r.choice(test_set, size=400, replace=False), maxlen=ml,tokenmaker=tm)
    newhistory = testmodel.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=32,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [32]:
history = None

In [ ]:
for i in range(20):
    print(i)
    history = training_round(history)

0
Train on 2000 samples, validate on 400 samples
Epoch 1/5
2000/2000 [==============================] - 44s - loss: 1.4378 - categorical_accuracy: 0.6869 - val_loss: 1.5296 - val_categorical_accuracy: 0.6629
Epoch 2/5
2000/2000 [==============================] - 43s - loss: 1.3764 - categorical_accuracy: 0.6920 - val_loss: 1.4511 - val_categorical_accuracy: 0.6741
Epoch 3/5
2000/2000 [==============================] - 43s - loss: 1.3099 - categorical_accuracy: 0.7008 - val_loss: 1.3845 - val_categorical_accuracy: 0.6846
Epoch 4/5
2000/2000 [==============================] - 43s - loss: 1.2866 - categorical_accuracy: 0.7049 - val_loss: 1.3736 - val_categorical_accuracy: 0.6872
Epoch 5/5
2000/2000 [==============================] - 43s - loss: 1.2631 - categorical_accuracy: 0.7068 - val_loss: 1.3212 - val_categorical_accuracy: 0.6992
1
Train on 2000 samples, validate on 400 samples
Epoch 6/10
2000/2000 [==============================] - 44s - loss: 1.2552 - categorical_accuracy: 0.7121 -

In [20]:
testmodel.fit(x=train.x, y=train.y,
                            epochs=5, batch_size=32,
                            validation_data=(test.x, test.y)
                            )

Train on 2000 samples, validate on 400 samples
Epoch 1/5
2000/2000 [==============================] - 46s - loss: 2.4730 - categorical_accuracy: 0.6573 - val_loss: 1.8280 - val_categorical_accuracy: 0.6826
Epoch 2/5
2000/2000 [==============================] - 44s - loss: 1.7381 - categorical_accuracy: 0.6790 - val_loss: 1.6597 - val_categorical_accuracy: 0.6852
Epoch 3/5
2000/2000 [==============================] - 43s - loss: 1.5909 - categorical_accuracy: 0.6829 - val_loss: 1.5535 - val_categorical_accuracy: 0.6862
Epoch 4/5
2000/2000 [==============================] - 44s - loss: 1.5106 - categorical_accuracy: 0.6854 - val_loss: 1.4956 - val_categorical_accuracy: 0.6894
Epoch 5/5
2000/2000 [==============================] - 43s - loss: 1.4549 - categorical_accuracy: 0.6878 - val_loss: 1.4695 - val_categorical_accuracy: 0.6864


In [32]:
testmodel.fit(x=train.x, y=train.y,
                            epochs=150, batch_size=32,
                            validation_data=(test.x, test.y)
                            )

Train on 2000 samples, validate on 400 samples
Epoch 1/150
2000/2000 [==============================] - 22s - loss: 0.4866 - categorical_accuracy: 0.8453 - val_loss: 2.0084 - val_categorical_accuracy: 0.6918
Epoch 2/150
2000/2000 [==============================] - 22s - loss: 0.3584 - categorical_accuracy: 0.8887 - val_loss: 2.0546 - val_categorical_accuracy: 0.6971
Epoch 3/150
2000/2000 [==============================] - 23s - loss: 0.3081 - categorical_accuracy: 0.9097 - val_loss: 2.1250 - val_categorical_accuracy: 0.6963
Epoch 4/150
2000/2000 [==============================] - 23s - loss: 0.2800 - categorical_accuracy: 0.9214 - val_loss: 2.1742 - val_categorical_accuracy: 0.6964
Epoch 5/150
2000/2000 [==============================] - 23s - loss: 0.2598 - categorical_accuracy: 0.9292 - val_loss: 2.2207 - val_categorical_accuracy: 0.6980
Epoch 6/150
2000/2000 [==============================] - 24s - loss: 0.2411 - categorical_accuracy: 0.9364 - val_loss: 2.2806 - val_categorical_accu

2000/2000 [==============================] - 26s - loss: 0.0252 - categorical_accuracy: 0.9963 - val_loss: 3.1772 - val_categorical_accuracy: 0.6865
Epoch 102/150
2000/2000 [==============================] - 26s - loss: 0.0218 - categorical_accuracy: 0.9966 - val_loss: 3.2042 - val_categorical_accuracy: 0.6858
Epoch 103/150
2000/2000 [==============================] - 26s - loss: 0.0205 - categorical_accuracy: 0.9967 - val_loss: 3.2280 - val_categorical_accuracy: 0.6862
Epoch 104/150
2000/2000 [==============================] - 26s - loss: 0.0200 - categorical_accuracy: 0.9967 - val_loss: 3.2487 - val_categorical_accuracy: 0.6859
Epoch 105/150
2000/2000 [==============================] - 27s - loss: 0.0197 - categorical_accuracy: 0.9966 - val_loss: 3.2704 - val_categorical_accuracy: 0.6857
Epoch 106/150
2000/2000 [==============================] - 24s - loss: 0.0187 - categorical_accuracy: 0.9967 - val_loss: 3.2880 - val_categorical_accuracy: 0.6851
Epoch 107/150
2000/2000 [===========